In [1]:
import numpy as np
import pandas as pd
import math 
from pymoo.factory import get_problem
import GA
import statistics


pf = get_problem("truss2d").pareto_front()
phi1= pf[:,0].reshape(-1,1)
phi2= pf[:,1].reshape(-1,1)
x1=phi1/8.944
x2=2*x1
m =pf.shape[0]

In [2]:
def cnf(pop):
    pop=pd.DataFrame(pop)
    pop=pop.drop([1,3,5,7,8],axis=1)
    cnf=pop.to_numpy()
    return cnf

In [3]:
def niched_tournament_selection(pop,fitness):
    
    fitness=fitness.reshape((fitness.shape[0],1))
    pop_=np.array(np.concatenate((pop,fitness),axis=1))
    pop1_=np.array(pop_.copy())
    np.random.shuffle(pop1_)
    pop1=pop1_[:,:-1]
    parents=np.empty((pop.shape))
    pop2_=np.empty((pop1_.shape))
    a=cnf(pop)
    b=cnf(pop1)
    for i in range(pop_size):
        for j in range(pop_size):
            if (a[j]==b[i]).all():
                pop2_[i]=pop_[j]
        if pop1_[i,-1]>=pop2_[i,-1]:
            parents[i]=pop1_[i][:-1]
        else:
            parents[i]=pop2_[i][:-1]
            
    return parents

In [4]:
def fitness(pop):
    fitness_pop = np.empty([pop_size,1])
    fitness_pop[:]=np.nan
    
    d=pop[:,-1]
    d= d.astype(int)

    c= np.array((phi1**(pop[:,0]*pop[:,1]))*(phi2**(pop[:,2]*pop[:,3]))*(x1**(pop[:,4]*pop[:,5]))*(x2**(pop[:,6]*pop[:,7])))
    c.sort(axis=0)
    X=(c[-1,:]-c[0,:])/d
    
    for j in range(pop_size):
        C=0
        U=0
        cv=0
        flag=1
        minim=0
        temp=[]
        for i in range(d[j]):
            
            minim=c[:,j].min()
   
            points = c[:,j][(c[:,j]>=(minim+i*X[j]))*(c[:,j]<(minim+(i+1)*X[j]+0.00000000001))]
            count = points.size
            if count <= int(m/d[j]) : 
                U=U+count
                flag=1
                if len(temp)!=0:
                    cv+=statistics.stdev(temp)/statistics.mean(temp)
                    temp=[]
            else:
                temp.extend(points)
                if flag == 1:
                    C=C+1
                    flag=0
        
        b = [1,2,3,4]
        constr_violation = 0
        for i in range (len(b)):
            if 0.1 - abs(b[i])>0:
                constr_violation += 0.1 - abs(b[i])
        fitness_pop[j]=-(C+U+(100*cv))+1000*constr_violation   
    
    return fitness_pop

In [5]:
import random
pop_size=400
n_gen=1000

pop= np.empty([pop_size,9])
for i in range(pop_size):
    for j in range(8):
        if j%2==0:
            pop[i][j]=random.randint(0,1)
        else:
            if random.random()>=0.5:
                pop[i][j]=random.uniform(-1,-0.1)
            else :
                pop[i][j]=random.uniform(0.1,1)
                
    if (pop[i][0:7:2]==np.array([0,0,0,0])).all()==True:
        pop[i][0:7:2][random.randint(0,3)]=1

    pop[i][8]=random.randint(1,m)
    
    bmax=np.max(abs(pop[i][1:8:2]*pop[i][0:7:2]))
    pop[i][1:8:2]=pop[i][1:8:2]/bmax
    if np.min(pop[i][1:8:2]*pop[i][0:7:2])==-1:
        pop[i][1:8:2]=-pop[i][1:8:2]

        
    size=pop[:,1:8:2][abs(pop[:,1:8:2])>1].size
    if size!=0:
        t=np.append(np.random.uniform(-1,-0.1,int(size/2)),np.random.uniform(0.1,1,size-int(size/2)))
        np.random.shuffle(t)
        pop[:,1:8:2][abs(pop[:,1:8:2])>1]=t
        
f=fitness(pop)

In [6]:
for i in range(n_gen):
    
    parents=niched_tournament_selection(pop,f)  
    a=parents[:,0:7:2]
    b=parents[:,1:8:2]
    d=parents[:,-1]
    child=np.full((parents.shape),np.nan)
    #CROSSOVERS
    a=GA.sbx_int(a,1,10,0,1)
    b=GA.sbx_real(b,0.9,10)
    d=GA.sbx_int(d,1,10,1,1100)[:,0]
    
    #MUTATIONS
    child[:,0:7:2]=GA.mut_flip(a,0.15)
    child[:,1:8:2]=GA.mut_poly(b,50,-1,1,0.05)
    child[:,8]=GA.mut_int(d,50,1,1100,0.05)[:,0]
    
    for l in range(pop_size):
        
        if (child[l][0:7:2]==np.array([0,0,0,0])).all()==True:
            child[l][0:7:2][random.randint(0,3)]=1
        
        bmax=np.max(abs(child[l][1:8:2]*child[l][0:7:2]))
        child[l][1:8:2]=child[l][1:8:2]/bmax
        if np.min(child[l][1:8:2]*child[l][0:7:2])==-1:
            child[l][1:8:2]=-child[l][1:8:2]
        
    
            
    size=child[:,1:8:2][(abs(child[:,1:8:2])>1)+(abs(child[:,1:8:2])<0.1)].size
    if size!=0:
        t=np.append(np.random.uniform(-1,-0.1,int(size/2)),np.random.uniform(0.1,1,size-int(size/2)))
        np.random.shuffle(t)
        child[:,1:8:2][(abs(child[:,1:8:2])>1)+(abs(child[:,1:8:2])<0.1)]=t
        
    pop=child
    
    f=fitness(child)
    
    print("gen:",i+1,"=",child)

gen: 1 = [[ 1.00000000e+00  2.80930932e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  6.95000000e+02]
 [ 1.00000000e+00 -5.36562728e-01  1.00000000e+00 ...  1.00000000e+00
  -6.32067886e-01  4.11000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
  -4.19528633e-01  8.17000000e+02]
 ...
 [ 1.00000000e+00  9.60373041e-01  0.00000000e+00 ...  0.00000000e+00
   2.17045558e-01  9.29000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   4.74880701e-01  6.17000000e+02]
 [ 0.00000000e+00 -3.55991093e-01  0.00000000e+00 ...  0.00000000e+00
  -4.28932246e-01  4.20000000e+01]]
gen: 2 = [[ 0.00000000e+00 -8.08691308e-01  1.00000000e+00 ...  0.00000000e+00
   6.86960195e-01  3.97000000e+02]
 [ 1.00000000e+00 -3.13340767e-01  1.00000000e+00 ...  0.00000000e+00
   5.41737053e-01  6.13000000e+02]
 [ 0.00000000e+00  8.35268834e-01  1.00000000e+00 ...  0.00000000e+00
  -8.28919797e-01  6.10000000e+02]
 ...
 [ 0.00000000e+00  7.75509

gen: 14 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -6.89949334e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  1.00000000e+00
  -3.52866830e-01  5.52000000e+02]
 [ 0.00000000e+00  8.05130512e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 ...
 [ 0.00000000e+00  8.99444892e-01  0.00000000e+00 ...  0.00000000e+00
   5.26227882e-01  5.64000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
  -2.31990662e-01  5.65000000e+02]
 [ 1.00000000e+00 -7.74509109e-01  1.00000000e+00 ...  0.00000000e+00
  -4.30679676e-01  5.65000000e+02]]
gen: 15 = [[ 1.00000000e+00 -3.19084486e-01  1.00000000e+00 ...  0.00000000e+00
  -7.64832928e-01  5.65000000e+02]
 [ 1.00000000e+00  9.59011611e-01  1.00000000e+00 ...  1.00000000e+00
  -3.30756461e-01  5.63000000e+02]
 [ 0.00000000e+00  7.04271124e-01  1.00000000e+00 ...  1.00000000e+00
  -8.45627682e-01  5.88000000e+02]
 ...
 [ 0.00000000e+00 -1.066

gen: 27 = [[ 1.00000000e+00 -4.56180332e-01  1.00000000e+00 ...  0.00000000e+00
   2.77252908e-01  5.54000000e+02]
 [ 1.00000000e+00 -7.96179012e-01  1.00000000e+00 ...  0.00000000e+00
   9.75693078e-01  5.54000000e+02]
 [ 0.00000000e+00 -2.92526674e-01  0.00000000e+00 ...  0.00000000e+00
  -8.27275292e-01  5.54000000e+02]
 ...
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   6.30494111e-01  5.54000000e+02]
 [ 0.00000000e+00  4.15826548e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.57000000e+02]
 [ 0.00000000e+00 -8.63983805e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.51000000e+02]]
gen: 28 = [[ 1.00000000e+00 -5.00481635e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 0.00000000e+00 -7.90415421e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   6.10977363e-01  5.54000000e+02]
 ...
 [ 0.00000000e+00  3.359

gen: 40 = [[ 1.00000000e+00  5.79074818e-01  1.00000000e+00 ...  1.00000000e+00
  -2.54483074e-01  5.54000000e+02]
 [ 0.00000000e+00 -4.55375976e-01  1.00000000e+00 ...  0.00000000e+00
  -6.73227704e-01  5.54000000e+02]
 [ 0.00000000e+00 -2.44001260e-01  1.00000000e+00 ...  0.00000000e+00
  -4.26887262e-01  5.54000000e+02]
 ...
 [ 0.00000000e+00  7.41769288e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 0.00000000e+00  7.83689991e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 0.00000000e+00  7.09664326e-01  1.00000000e+00 ...  1.00000000e+00
   2.78806073e-01  5.54000000e+02]]
gen: 41 = [[ 1.00000000e+00  7.38810618e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 0.00000000e+00 -7.57834272e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 [ 0.00000000e+00 -5.72431212e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 ...
 [ 0.00000000e+00 -5.196

gen: 53 = [[ 0.00000000e+00 -7.07512246e-01  0.00000000e+00 ...  0.00000000e+00
   1.14557606e-01  5.54000000e+02]
 [ 1.00000000e+00  8.07267135e-01  1.00000000e+00 ...  0.00000000e+00
   7.57719423e-01  5.53000000e+02]
 [ 0.00000000e+00  5.84489205e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]
 ...
 [ 0.00000000e+00 -9.32291253e-01  1.00000000e+00 ...  0.00000000e+00
  -6.04972009e-01  5.54000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
  -2.23316273e-01  5.53000000e+02]
 [ 1.00000000e+00  8.77373943e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.54000000e+02]]
gen: 54 = [[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   4.63042828e-01  5.53000000e+02]
 [ 0.00000000e+00  3.18228173e-01  1.00000000e+00 ...  0.00000000e+00
   5.72892509e-01  5.54000000e+02]
 [ 0.00000000e+00 -1.67139050e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.53000000e+02]
 ...
 [ 0.00000000e+00  8.644

...

gen: 937 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -6.60874489e-01  5.52000000e+02]
 [ 0.00000000e+00  1.96924448e-01  0.00000000e+00 ...  0.00000000e+00
  -9.55759440e-01  5.52000000e+02]
 [ 1.00000000e+00  7.03064206e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.55000000e+02]
 ...
 [ 1.00000000e+00 -4.90046277e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 0.00000000e+00 -4.42849670e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 0.00000000e+00 -6.12247562e-01  0.00000000e+00 ...  1.00000000e+00
   2.40163688e-01  5.52000000e+02]]
gen: 938 = [[ 0.00000000e+00  6.45340596e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -9.72886036e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  1.00000000e+00
  -2.11979302e-01  5.52000000e+02]
 ...
 [ 0.00000000e+00 -2.6

gen: 950 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   7.11324466e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -6.93856466e-01  5.52000000e+02]
 [ 1.00000000e+00  8.08580978e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 ...
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
  -8.84295912e-01  5.52000000e+02]
 [ 0.00000000e+00 -9.87601577e-01  1.00000000e+00 ...  0.00000000e+00
   6.01321574e-01  5.52000000e+02]
 [ 0.00000000e+00 -8.86646141e-01  0.00000000e+00 ...  1.00000000e+00
   2.61799418e-01  5.52000000e+02]]
gen: 951 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -2.66331584e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  1.00000000e+00
   8.18835178e-01  5.52000000e+02]
 [ 0.00000000e+00  1.20056357e-01  0.00000000e+00 ...  0.00000000e+00
   3.06277716e-01  5.52000000e+02]
 ...
 [ 0.00000000e+00 -9.3

gen: 963 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   8.37099168e-01  5.52000000e+02]
 [ 1.00000000e+00 -3.02583612e-01  1.00000000e+00 ...  0.00000000e+00
   8.66713722e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -6.64698905e-01  5.52000000e+02]
 ...
 [ 1.00000000e+00 -4.05235074e-01  1.00000000e+00 ...  0.00000000e+00
   7.66910047e-01  5.52000000e+02]
 [ 1.00000000e+00 -2.18742153e-01  0.00000000e+00 ...  0.00000000e+00
  -8.31888991e-01  5.52000000e+02]
 [ 1.00000000e+00 -2.67945110e-01  1.00000000e+00 ...  1.00000000e+00
   7.14143501e-01  5.52000000e+02]]
gen: 964 = [[ 0.00000000e+00 -6.32014299e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 0.00000000e+00  3.50186918e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 1.00000000e+00 -7.66885733e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 ...
 [ 0.00000000e+00 -7.2

gen: 976 = [[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   8.03985541e-01  5.52000000e+02]
 [ 1.00000000e+00 -5.86279623e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 1.00000000e+00 -3.44664384e-01  0.00000000e+00 ...  1.00000000e+00
  -2.45735468e-01  5.53000000e+02]
 ...
 [ 1.00000000e+00 -5.06982088e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 0.00000000e+00 -2.86753994e-01  1.00000000e+00 ...  1.00000000e+00
   2.54826938e-01  5.52000000e+02]
 [ 0.00000000e+00 -9.81711722e-01  1.00000000e+00 ...  1.00000000e+00
   6.95588767e-01  5.52000000e+02]]
gen: 977 = [[ 1.00000000e+00  8.65359554e-01  1.00000000e+00 ...  1.00000000e+00
   3.97073116e-01  5.52000000e+02]
 [ 0.00000000e+00  3.00615333e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  5.52000000e+02]
 [ 0.00000000e+00 -4.11508622e-01  0.00000000e+00 ...  1.00000000e+00
  -2.23485567e-01  5.52000000e+02]
 ...
 [ 0.00000000e+00 -8.3

gen: 989 = [[ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.11258365e-01  5.52000000e+02]
 [ 0.00000000e+00  5.44664281e-01  0.00000000e+00 ...  0.00000000e+00
  -9.78346269e-01  5.52000000e+02]
 [ 1.00000000e+00 -5.52066525e-01  0.00000000e+00 ...  0.00000000e+00
  -5.05219685e-01  5.52000000e+02]
 ...
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   2.05203761e-01  5.52000000e+02]
 [ 0.00000000e+00  4.13946355e-01  0.00000000e+00 ...  0.00000000e+00
   8.53699717e-01  5.52000000e+02]
 [ 0.00000000e+00  7.36209724e-01  0.00000000e+00 ...  0.00000000e+00
   8.92715263e-01  5.52000000e+02]]
gen: 990 = [[ 0.00000000e+00 -5.49401570e-01  1.00000000e+00 ...  1.00000000e+00
  -1.61451316e-01  5.37000000e+02]
 [ 0.00000000e+00  1.56750988e-01  1.00000000e+00 ...  1.00000000e+00
   8.57884229e-01  5.52000000e+02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  1.00000000e+00
  -7.22162854e-01  5.54000000e+02]
 ...
 [ 0.00000000e+00  9.2

In [7]:
#REDEFINING CLUSTERING CRITERION

pop=child
d=pop[:,-1]
d= d.astype(int)
unclustered_points=np.full((pop_size,1),np.nan)
clusters=np.full((pop_size,1),np.nan)
c =(phi1**(pop[:,0]*pop[:,1]))*(phi2**(pop[:,2]*pop[:,3]))*(x1**(pop[:,4]*pop[:,5]))*(x2**(pop[:,6]*pop[:,7]))                          
c.sort(axis=0)
X=(c[-1,:]-c[0,:])/d
for j in range(pop_size):
    C=0
    U=0
    cv=0
    flag=1
    minim=0
    temp=[]
    for i in range(d[j]):

        minim=c[:,j].min()

        points = c[:,j][(c[:,j]>=(minim+i*X[j]))*(c[:,j]<(minim+(i+1)*X[j]+0.00000000001))]
        count = points.size
        if count <= int(m/d[j])+3 : 
            U=U+count
            flag=1
            if len(temp)!=0:
                cv+=statistics.stdev(temp)/statistics.mean(temp)
                temp=[]
        else:
            temp.extend(points)
            if flag == 1:
                C=C+1
                flag=0
    

    unclustered_points[j]=U
    clusters[j]=C

In [8]:
unclustered_points

array([[1100.],
       [ 579.],
       [ 374.],
       [ 416.],
       [1100.],
       [ 901.],
       [1095.],
       [ 788.],
       [1100.],
       [ 171.],
       [ 730.],
       [1070.],
       [1100.],
       [ 416.],
       [1100.],
       [1100.],
       [ 674.],
       [1100.],
       [ 297.],
       [1019.],
       [1100.],
       [ 853.],
       [1064.],
       [1100.],
       [1059.],
       [ 416.],
       [ 416.],
       [1100.],
       [ 918.],
       [ 262.],
       [ 571.],
       [1017.],
       [ 353.],
       [ 416.],
       [1100.],
       [1100.],
       [1100.],
       [1100.],
       [ 882.],
       [1100.],
       [1100.],
       [ 980.],
       [1100.],
       [1019.],
       [ 827.],
       [1100.],
       [1100.],
       [ 524.],
       [1059.],
       [1100.],
       [ 636.],
       [1100.],
       [1100.],
       [ 943.],
       [ 416.],
       [1100.],
       [ 588.],
       [1100.],
       [ 887.],
       [ 892.],
       [1050.],
       [ 882.],
       [

In [33]:
a=pd.DataFrame((1100-unclustered_points)/11)
b=a[a>=70]
b.dropna(inplace=True)
b

,0
9,84.454545
18,73.000000
29,76.181818
74,92.545455
85,77.090909
91,79.181818
122,76.272727
128,100.000000
136,92.363636
138,71.636364


POINTS WITH STATISTICAL SIGNIFIACANC

In [17]:
pop[9]

array([  0.        ,   0.84551368,   1.        ,   1.        ,
         0.        ,   0.97674822,   1.        ,   0.99311373,
       556.        ])

In [18]:
pop[74]

array([  1.        ,   0.99647626,   1.        ,   1.        ,
         0.        ,   0.94139721,   0.        ,  -0.71786868,
       552.        ])

In [19]:
pop[128]

array([ 0.00000000e+00,  7.50429377e-01,  0.00000000e+00,  4.37576385e-01,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00, -1.00000000e+00,
        5.52000000e+02])

In [20]:
pop[136]

array([  1.        ,   0.99776178,   1.        ,   1.        ,
         0.        ,   0.94588217,   0.        ,  -0.84012554,
       552.        ])

In [21]:
pop[303]

array([  1.        ,   0.99776178,   1.        ,   1.        ,
         0.        ,   0.94588217,   0.        ,  -0.84012554,
       552.        ])